In [1]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

# Load config values
with open(r"config.json") as config_file:
    config_details = json.load(config_file)

client = OpenAI()

model_name = config_details["MODEL_NAME"]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)
print(completion.choices[0].message)


ChatCompletionMessage(content='Code within a code,\nLooping through the mirrored depths,\nInfinite unfolds.', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [2]:
from simple_robot import SimpleRobot
import re
import ast
robot = SimpleRobot()

def list_functions_with_docs(obj):
    return [(attr, getattr(obj, attr).__doc__) for attr in dir(obj) if callable(getattr(obj, attr)) and not attr.startswith("__")]

robot_functions = list_functions_with_docs(robot)
print(robot_functions)

tools = []

def string_to_dict(s):
    s = s.strip()
    s = '{' + s + '}'
    
    # Replace single quotes with double quotes for JSON compatibility
    s = re.sub(r"'", '"', s)
    
    # Replace Python-style True/False with JSON-style true/false
    s = s.replace('True', 'true').replace('False', 'false')
    
    try:
        # Use ast.literal_eval to safely evaluate the string
        return ast.literal_eval(s)
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing string: {e}")
        return None

for fun_name, docstr in robot_functions:
    fun_desc = {}
    if docstr:
        fun_desc = string_to_dict(docstr)
    tools += [{
                "type": "function",
                "function": {
                    "name": fun_name
                }
              }]
    tools[-1]["function"].update(fun_desc)
tools

[('all_leds_off', '\n            "description": "Turn all LEDs off"\n        '), ('animate_police_lights', '\n            "description": "Cycle the group1 and group2 LEDs between color1 and color2 to give the effect of police lights.  Alternate the group1 and group2 LEDs every sleeptime seconds. Animate for duration seconds.  If duration is None animate for forever.",\n            "parameters": {\n                "type": "object",\n                "properties": {\n                    "color1": {\n                        "type": "string",\n                        "enum": ["BLACK", "RED", "GREEN", "AMBER", "ORANGE", "YELLOW"]\n                    },\n                    "color2": {\n                        "type": "string",\n                        "enum": ["BLACK", "RED", "GREEN", "AMBER", "ORANGE", "YELLOW"]\n                    },\n                    "group1": {\n                        "type": "string",\n                        "description": "Which LED to change the color.",\n     

[{'type': 'function',
  'function': {'name': 'all_leds_off', 'description': 'Turn all LEDs off'}},
 {'type': 'function',
  'function': {'name': 'animate_police_lights',
   'description': 'Cycle the group1 and group2 LEDs between color1 and color2 to give the effect of police lights.  Alternate the group1 and group2 LEDs every sleeptime seconds. Animate for duration seconds.  If duration is None animate for forever.',
   'parameters': {'type': 'object',
    'properties': {'color1': {'type': 'string',
      'enum': ['BLACK', 'RED', 'GREEN', 'AMBER', 'ORANGE', 'YELLOW']},
     'color2': {'type': 'string',
      'enum': ['BLACK', 'RED', 'GREEN', 'AMBER', 'ORANGE', 'YELLOW']},
     'group1': {'type': 'string',
      'description': 'Which LED to change the color.',
      'enum': ['LEFT', 'RIGHT', 'BOTH']},
     'group2': {'type': 'string',
      'description': 'Which LED to change the color.',
      'enum': ['LEFT', 'RIGHT', 'BOTH']}},
    'required': []}}},
 {'type': 'function',
  'function

In [42]:
messages = [{"role": "system", "content": "You control a legon mindstorms robot with evdev3 by calling different functions that execute commands on the robot"},
            {"role": "user", "content": "make the right light green"}]

# Call the model with the user query (messages) and the functions defined in the functions parameter
response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=tools,
    tool_choice="auto",
)


In [43]:
import xmlrpc.client
import inspect

s = xmlrpc.client.ServerProxy('http://192.168.178.145:8000', allow_none=True)

available_functions = {}
for fun_name, _ in robot_functions:
    available_functions[fun_name] = getattr(s, fun_name)

chosen_fn_name = response.choices[0].message.tool_calls[0].function.name
function_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)

args = []
for arg in list(inspect.signature(getattr(robot, fun_name)).parameters):
    if arg in function_args.keys():
        args += [function_args[arg]]

available_functions[chosen_fn_name](*args)

In [30]:
args

['LEFT', 'GREEN']

In [17]:
function_args

{'group': 'LEFT', 'color': 'GREEN'}

In [27]:
import inspect
list(inspect.signature(robot.set_led_color).parameters)

['group', 'color', 'pct']